In [7]:
# Código pronto para Google Colab - versão auto-adaptável
# =========================================
# Execução:
# 1) Cole este script em uma célula do Colab
# 2) Execute a célula e, quando solicitado, faça upload dos arquivos .csv
# 3) Ao terminar, um arquivo ZIP será disponibilizado para download
# =========================================

import os
import io
import shutil
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [8]:
# ---------------------------
# CONFIGURAÇÕES
# ---------------------------
EMPIRICAL_THRESHOLD = 100.0   # limiar empírico
RMS_WIN_MS = 20               # janela RMS em ms
METRIC_SHIFT_S = 0.5          # deslocamento de 0.5s para média
OUTPUT_ROOT = "/content/Gráficos_PNG"  # pasta raiz no Colab

In [9]:
# ---------------------------
# FUNÇÕES AUXILIARES
# ---------------------------
def ms_to_samples(ms, dt_ms):
    return int(round(ms / dt_ms))

def calc_envelope_rms(signal, win_samples):
    if win_samples <= 1:
        return np.abs(signal)
    kernel = np.ones(win_samples) / win_samples
    sq = signal.astype(float) ** 2
    env = np.sqrt(np.convolve(sq, kernel, mode='same'))
    return env

def detect_onset_offset(env, threshold):
    above = np.where(env > threshold)[0]
    if above.size == 0:
        return None, None
    return int(above[0]), int(above[-1])

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


In [10]:
# ---------------------------
# UPLOAD
# ---------------------------
print("📂 Faça o upload dos arquivos CSV (pode selecionar vários).")
uploaded = files.upload()

if not uploaded:
    raise SystemExit("Nenhum arquivo enviado.")

ensure_dir(OUTPUT_ROOT)

summary = []

📂 Faça o upload dos arquivos CSV (pode selecionar vários).


Saving dados (3)_filtrado.csv to dados (3)_filtrado.csv
Saving dados (4)_filtrado.csv to dados (4)_filtrado.csv
Saving dados (5)_filtrado.csv to dados (5)_filtrado.csv
Saving dados (6)_filtrado.csv to dados (6)_filtrado.csv
Saving dados (7)_filtrado.csv to dados (7)_filtrado.csv
Saving dados (8)_filtrado.csv to dados (8)_filtrado.csv
Saving dados (9)_filtrado.csv to dados (9)_filtrado.csv
Saving dados (26)_filtrado.csv to dados (26)_filtrado.csv
Saving dados (27)_filtrado.csv to dados (27)_filtrado.csv
Saving dados (28)_filtrado.csv to dados (28)_filtrado.csv
Saving dados (29)_filtrado.csv to dados (29)_filtrado.csv
Saving dados (30)_filtrado.csv to dados (30)_filtrado.csv
Saving dados (1)_filtrado.csv to dados (1)_filtrado.csv
Saving dados (2)_filtrado.csv to dados (2)_filtrado.csv
Saving dados (10)_filtrado.csv to dados (10)_filtrado.csv
Saving dados (11)_filtrado.csv to dados (11)_filtrado.csv
Saving dados (12)_filtrado.csv to dados (12)_filtrado.csv
Saving dados (13)_filtrado.csv t

In [11]:
# ---------------------------
# LOOP PRINCIPAL
# ---------------------------
for filename, filebytes in uploaded.items():
    try:
        print(f"\n🔁 Processando arquivo: {filename}")
        df = pd.read_csv(io.BytesIO(filebytes))

        # Detecta colunas
        colunas = [c.strip() for c in df.columns]
        tempo_col = colunas[0]          # primeira coluna = tempo
        sinais_cols = colunas[1:]       # restantes = sinais

        tempo_ms = df[tempo_col].values.astype(float)

        # calcular dt
        diffs = np.diff(tempo_ms)
        dt_ms = float(np.median(diffs))
        fs = 1000.0 / dt_ms
        win_samples = ms_to_samples(RMS_WIN_MS, dt_ms)
        shift_samples = int(round(METRIC_SHIFT_S * fs))

        # pasta de saída
        base_name = os.path.splitext(os.path.basename(filename))[0]
        out_dir = os.path.join(OUTPUT_ROOT, base_name)
        ensure_dir(out_dir)

        metrics = []

        # --- gráficos individuais ---
        for col in sinais_cols:
            sig = df[col].values.astype(float)
            env = calc_envelope_rms(sig, win_samples)

            start, end = detect_onset_offset(env, EMPIRICAL_THRESHOLD)

            media = rms = None
            if start is not None and end is not None:
                s_shift = start + shift_samples
                if s_shift >= end:
                    s_shift = start
                trecho = sig[s_shift:end]
                if trecho.size > 0:
                    media = float(np.mean(trecho))
                    rms = float(np.sqrt(np.mean(trecho**2)))

            # plot
            plt.figure(figsize=(10,5))
            plt.plot(tempo_ms, sig, label=f'{col}', color='#1f77b4')
            plt.plot(tempo_ms, env, label='Envelope RMS', color='orange')
            plt.axhline(EMPIRICAL_THRESHOLD, color='red', linestyle='--', label=f'Limiar {EMPIRICAL_THRESHOLD}')
            if start and end:
                plt.axvline(tempo_ms[start], color='green', linestyle=':', label='Início')
                plt.axvline(tempo_ms[end], color='red', linestyle=':', label='Fim')
                if media is not None:
                    plt.hlines(media, tempo_ms[start], tempo_ms[end],
                               colors='blue', linestyles='--', label=f'Média: {media:.1f}')
            plt.title(f'{col} - {base_name}')
            plt.xlabel('Tempo (ms)')
            plt.ylabel('Amplitude')
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            outpath = os.path.join(out_dir, f'{col}_{base_name}_endpoint.png')
            plt.savefig(outpath, dpi=200)
            plt.close()

            metrics.append({
                "arquivo": filename,
                "coluna": col,
                "inicio_ms": tempo_ms[start] if start else '',
                "fim_ms": tempo_ms[end] if end else '',
                "media": media if media else '',
                "rms": rms if rms else ''
            })

        # --- gráfico combinado ---
        plt.figure(figsize=(10,5))
        for col in sinais_cols:
            plt.plot(tempo_ms, df[col].values, label=col)
        plt.title(f'Combinado - {base_name}')
        plt.xlabel('Tempo (ms)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        outpath_comb = os.path.join(out_dir, f'Combinado_{base_name}_endpoint.png')
        plt.savefig(outpath_comb, dpi=200)
        plt.close()

        # salvar métricas individuais
        metrics_df = pd.DataFrame(metrics)
        metrics_df.to_csv(os.path.join(out_dir, f'metricas_{base_name}.csv'), index=False)
        summary.extend(metrics)

        print(f"✅ Processado e salvo em {out_dir}")

    except Exception as e:
        print(f"❌ Erro ao processar {filename}: {e}")



🔁 Processando arquivo: dados (3)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (3)_filtrado

🔁 Processando arquivo: dados (4)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (4)_filtrado

🔁 Processando arquivo: dados (5)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (5)_filtrado

🔁 Processando arquivo: dados (6)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (6)_filtrado

🔁 Processando arquivo: dados (7)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (7)_filtrado

🔁 Processando arquivo: dados (8)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (8)_filtrado

🔁 Processando arquivo: dados (9)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (9)_filtrado

🔁 Processando arquivo: dados (26)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (26)_filtrado

🔁 Processando arquivo: dados (27)_filtrado.csv
✅ Processado e salvo em /content/Gráficos_PNG/dados (2

In [12]:
# ---------------------------
# ZIP FINAL
# ---------------------------
summary_df = pd.DataFrame(summary)
summary_csv = os.path.join(OUTPUT_ROOT, f'resumo_metricas_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv')
summary_df.to_csv(summary_csv, index=False)

zip_name = f'Graficos_PNG_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
shutil.make_archive(f'/content/{zip_name}', 'zip', OUTPUT_ROOT)
print("⬇️ Iniciando download do ZIP...")
files.download(f'/content/{zip_name}.zip')

⬇️ Iniciando download do ZIP...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>